# **Juegos de palabras**

Para completar nuestros proyectos, crearemos divertidos juegos de palabras que pueden ser utilizados para el disfrute y el aprendizaje de las lenguas. Estos proyectos no deberían ser demasiado complejos técnicamente, pero te permitirán usar tus habilidades de Python de una manera creativa y divertida.

## **Descifrador de palabras**
Para el primer juego, vamos a proporcionar al azar una palabra donde las letras se mezclan, y el usuario tiene que descifrar la palabra.

### Preparación de datos
Primero, cargaremos el corpus. Para este proyecto, usaremos un corpus del latín (pero te invitamos a usar el corpus del Inglés, Uspanteko o el tuyo propio!

In [ ]:
# Load our data
import util

# REPLACE WITH YOUR CORPUS DIRECTORY
corpus = util.load_raw_text(corpus_directory="../../corpora/lat")

#### **Ejercicio 1**
Necesitamos un conjunto de todas las palabras únicas en el corpus. Siguiendo el enfoque utilizado en los proyectos del corrector ortográfico y el predictor de texto, haz lo siguiente:
1. Quitar los acentos
2. Hacer el texto en minúscula
3. Tokenizar
4. Crear y llenar un conjunto llamado `lexicon` con todas las palabras únicas.

Puedes usar cualquier función en el archivo `util.py`.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">import re

corpus = util.strip_accents(corpus)
corpus = corpus.lower()

def tokenize(text: str):
    return re.findall(word_regex, text)

words = tokenize(corpus)

lexicon = set()
for word in words:
    lexicon.add(word)</code></pre></details>

In [ ]:
word_regex = r"[a-z]+"

# TODO: See above!

print(len(lexicon), "unique words")

A continuación, vamos a filtrar cualquier palabra de menos de 4 letras (ya que serían demasiado fáciles) y de más de 8 letras (pues serían demasiado difíciles).

In [ ]:
filtered_lexicon = set()

for word in lexicon:
    if 4 <= len(word) <= 8:
        filtered_lexicon.add(word)

print(len(filtered_lexicon))

### Mezclando y seleccionando una palabra aleatoria
Vamos a construir una función que elija una palabra aleatoria y la mezcle.

In [ ]:
import random

def random_scramble(lexicon: set):
    lexicon = list(lexicon)
    
    word = random.choice(lexicon)
    
    # Turn the word into a list of characters 
    word_chars = list(word)
    
    # Shuffle those characters
    random.shuffle(word_chars)
    
    # Re-join the characters into a string
    shuffled = ''.join(word_chars)
    
    return {'shuffled': shuffled, 'original': word}

random_scramble(filtered_lexicon)

### Construir una aplicación
Por último, usamos esta función para construir un bucle donde el usuario adivina una palabra, y la aplicación les dice si era correcta o no.

In [ ]:
import gradio as gr
from typing import Tuple

def scrambler_game(current_word, guess: str):
    """
    If `guess` is the correct word, return 'Correct' and pick a new word. Otherwise, return 'Incorrect'
    Returns (correct_label, scrambled_word, current_word)
    """
    if guess == current_word['original']:
        current_word = random_scramble(filtered_lexicon)
        return ('Correct', current_word['shuffled'], current_word)
    else:
        return ('Incorrect', current_word['shuffled'], current_word)
    

def new_word():
    current_word = random_scramble(filtered_lexicon)
    return ('', current_word['shuffled'], current_word)
                
                
with gr.Blocks(theme=gr.themes.Soft(), title="Latin Unscramble") as unscramble:
    # Start with some initial word
    current_word = gr.State(random_scramble(filtered_lexicon))
    
    gr.Markdown("# Latin Unscramble")
    
    # Notice how we set the initial value based on the State
    scrambled_textbox = gr.Textbox(label="Scrambled", interactive=False, value=current_word.value['shuffled'])
    
    guess_textbox = gr.Textbox(label="Guess")
    guess_button = gr.Button(value="Submit")
    
    new_word_button = gr.Button(value="New Word")
    
    output_textbox = gr.Textbox(label="Result", interactive=False)
    
    guess_button.click(fn=scrambler_game, inputs=[current_word, guess_textbox], outputs=[output_textbox, scrambled_textbox, current_word])
    new_word_button.click(fn=new_word, inputs=[], outputs=[output_textbox, scrambled_textbox, current_word])
    
unscramble.launch(share=True)

### **Ejercicio 2**
Modifica el juego de descifrar con una o más de las siguientes características:
- Añade la habilidad de obtener una pista, como la primera letra de la palabra sin cifrar
- Dale - al usuario un número limitado de intentos para adivinar cada palabra
- Mantén un seguimiento de cuántas veces el usuario ha adivinado la palabra correcta
- Borrar la respuesta del usuario después de hacer clic en "Enviar"

In [ ]:
# TODO: Your improved word games app!

## **El ahorcado**

A continuación, vamos a implementar el juego del ahorcado en nuestra lengua de trabajo. En este juego, el ordenador elige una palabra aleatoria, y el jugador adivina una letra a la vez. El jugador sólo puede tener 6 letras incorrectas antes de perder el juego.

In [ ]:
def create_hangman_clue(word, guessed_letters):
    """
    Given a word and a list of letters, create the correct clue. 
    
    For instance, if the word is 'apple' and the guessed letters are 'a' and 'l', the clue should be 'a _ _ l _'
    """
    clue = ''
    for letter in word:
        if letter in guessed_letters:
            clue += letter + ' '
        else:
            clue += '_ '
    return clue
    

def pick_new_word(lexicon):
    lexicon = list(lexicon)
    
    return {
        'word': random.choice(lexicon),
        'guessed_letters': set(),
        'remaining_chances': 6
    }


def hangman_game(current_state, guess):
    """Update the current state based on the guess."""
    guess = guess.lower()
    
    if guess in current_state['guessed_letters'] or len(guess) > 1:
        # Illegal guess, do nothing
        return (current_state, 'Invalid guess')
    
    current_state['guessed_letters'].add(guess)
    
    if guess not in current_state['word']:
        # Wrong guess
        current_state['remaining_chances'] -= 1
        
        if current_state['remaining_chances'] == 0:
            # No more chances! New word
            current_state = pick_new_word(filtered_lexicon)
            return (current_state, 'You lose!')
        else:
            return (current_state, 'Wrong guess :(')
        
    else:
        # Right guess, check if there's any letters left
        for letter in current_state['word']:
            if letter not in current_state['guessed_letters']:
                # Still letters remaining
                return (current_state, 'Correct guess!')
        
        # If we made it here, there's no letters left.
        current_state = pick_new_word(filtered_lexicon)
        return (current_state, 'You win!')
    

def state_changed(current_state):
    clue = create_hangman_clue(current_state['word'], current_state['guessed_letters'])
    guessed_letters = current_state['guessed_letters']
    remaining_chances = current_state['remaining_chances']
    return (clue, guessed_letters, remaining_chances)


with gr.Blocks(theme=gr.themes.Soft(), title="Latin Hangman") as hangman:
    current_word = gr.State(pick_new_word(filtered_lexicon))
    
    gr.Markdown("# Latin Hangman")
    
    with gr.Row():
        current_word_textbox = gr.Textbox(label="Clue", interactive=False, value=create_hangman_clue(current_word.value['word'], current_word.value['guessed_letters']))
        guessed_letters_textbox = gr.Textbox(label="Guessed letters", interactive=False)
        remaining_chances_textbox = gr.Textbox(label="Remaining chances", interactive=False, value=6)
    
    guess_textbox = gr.Textbox(label="Guess")
    guess_button = gr.Button(value="Submit")
    
    output_textbox = gr.Textbox(label="Result", interactive=False)
    
    guess_button.click(fn=hangman_game, inputs=[current_word, guess_textbox], outputs=[current_word, output_textbox])\
                .then(fn=state_changed, inputs=[current_word], outputs=[current_word_textbox, guessed_letters_textbox, remaining_chances_textbox])
    
hangman.launch(share=True)

### **Ejercicio 3**
Modificar el juego del ahorcado con una o más de las siguientes características:
- Agrega la capacidad de obtener una pista, como una letra gratuita
- Cuando el jugador pierda una partida, muestra la palabra correcta
- Mantén un seguimiento de cuántas veces gana el jugador y pierde

In [ ]:
# TODO: Your improved hangman game

## **Construye tu propio juego**
Por último, para comprobar realmente cuánto has aprendido, elige algún juego de palabras y constrúyelo con un idioma de tu elección. Algunas ideas incluyen [Wordle](https://www.nytimes.com/games/wordle/index.html), que implica hacer tantas palabras sea posible a partir de un conjunto de letras, o la rueda de la Fortuna.

In [ ]:
# TODO: Your very own word game

**Resumen**
En este tutorial construimos algunos juegos lingüísticos fáciles y divertidos. Esto incluye:
- Selección y mezcla aleatoria de palabras al azar
- Creación de la lógica del juego
- Más experiencia con Gradio

Aunque no es tan técnicamente complejo como algunos de los otros proyectos, Los juegos de palabras son una gran manera de ayudar en el aprendizaje de las lenguas, que es fundamental para una lengua en peligro.

Si has llegado tan lejos, esperemos que hayas aprendido mucho sobre la construcción de tecnologías lingüística, incluso cuando los recursos son muy limitados. Esperamos que hayas disfrutado del BELT y puedas empezar a utilizar estas habilidades en el mundo real!